In [ ]:
import streamlit as st
import numpy as np
import akshare as ak
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import os


In [ ]:
# ----------------------------- 公募基金列表---------------------------##
df_fund = ak.fund_open_fund_rank_em(symbol="全部")
# 将列转换为数值，并将无法转换的值转换为NaN
df_fund['今年来'] = pd.to_numeric(df_fund['今年来'], errors='coerce')
df_fund['今年来'] = df_fund['今年来'].fillna(0)
# 剔除C类
df_fund = df_fund[~df_fund['基金简称'].str.contains('C', case=False)]
# 缩减输出列
df_fund = df_fund[['基金代码', '基金简称', '日期', '近1月', '近1年', '近3年', '今年来', '成立来', '手续费']]
# 今年来排序
df_fund_year = df_fund.sort_values('今年来', ascending=False)

# -----------------------------  今年来涨跌top10 and tail10 公募基金--------------------#
df_fund_year_top = df_fund_year.head(10)
df_fund_year_tail = df_fund_year.tail(10)

# -----------------------------   五、QDII专题----------------------------------------------#
# -----------------------------  今年来涨跌Top10 - QDII 基金---------------------------#

df_fund_qdii = ak.fund_open_fund_rank_em(symbol="QDII")
# 将列转换为数值，并将无法转换的值转换为NaN
df_fund_qdii['今年来'] = pd.to_numeric(df_fund_qdii['今年来'], errors='coerce')
df_fund_qdii['今年来'] = df_fund_qdii['今年来'].fillna(0)
# 剔除C类
df_fund_qdii = df_fund_qdii[~df_fund_qdii['基金简称'].str.contains('C', case=False)]
# 缩减输出列
df_fund_qdii = df_fund_qdii[['基金代码', '基金简称', '日期', '近1月', '近1年', '近3年', '今年来', '成立来', '手续费']]
# 排序输出
df_fund_qdii_year = df_fund_qdii.sort_values('今年来', ascending=False)

df_fund_qdii_year_top = df_fund_qdii_year.head(10)
df_fund_qdii_year_tail = df_fund_qdii_year.tail(10)